In [29]:
!pip install -q mealpy
import mealpy
mealpy.__version__

'2.5.4'

In [34]:
W_ENCODER = LabelEncoder()
W_ENCODER.fit(['uniform', 'distance'])
print(W_ENCODER.inverse_transform([0]))
print(W_ENCODER.inverse_transform([1]))

['distance']
['uniform']


In [35]:
def fitness_function(solution):
    N = int(solution[0])
    w_encoded = int(solution[1])
    w_decoded = W_ENCODER.inverse_transform([w_encoded])[0]

    knn = KNeighborsClassifier(n_neighbors= N, weights = w_decoded)
    knn.fit(X_train, y_train)
    y_predict = knn.predict(X_test)
    return metrics.accuracy_score(y_test, y_predict)

In [36]:
# Problem Defination
problem = {
    "fit_func": fitness_function,
    "lb": [2, 0],
    "ub": [100, 1],
    "minmax": "max",
}

In [37]:
# Termination Function
term_dict = {
    "max_early_stop": 50,
}

In [38]:
# Spotted Hyena Optimization
epoch = 500
pop_size = 50
h_factor = 5.0
N_tried = 10

model = OriginalSHO(epoch, pop_size, h_factor, N_tried)
best_position, best_fitness = model.solve(problem, termination=term_dict)
print(f"Solution: {best_position}, Fitness: {best_fitness}")

2023/09/10 10:41:28 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: Solving single objective optimization problem.
2023/09/10 10:41:30 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 1, Current best: 0.7489177489177489, Global best: 0.7489177489177489, Runtime: 2.02685 seconds
2023/09/10 10:41:32 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 2, Current best: 0.7489177489177489, Global best: 0.7489177489177489, Runtime: 1.98356 seconds
2023/09/10 10:41:34 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 3, Current best: 0.7489177489177489, Global best: 0.7489177489177489, Runtime: 1.72666 seconds
2023/09/10 10:41:36 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 4, Current best: 0.7489177489177489, Global best: 0.7489177489177489, Runtime: 1.78758 seconds
2023/09/10 10:41:38 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 5, Current best: 0.7489177489177489, Global best: 0.7489177489177489, Runtime

2023/09/10 10:43:06 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 46, Current best: 0.7489177489177489, Global best: 0.7489177489177489, Runtime: 1.37183 seconds
2023/09/10 10:43:08 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 47, Current best: 0.7489177489177489, Global best: 0.7489177489177489, Runtime: 1.83274 seconds
2023/09/10 10:43:10 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 48, Current best: 0.7489177489177489, Global best: 0.7489177489177489, Runtime: 1.69152 seconds
2023/09/10 10:43:11 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 49, Current best: 0.7489177489177489, Global best: 0.7489177489177489, Runtime: 1.26569 seconds
2023/09/10 10:43:13 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 50, Current best: 0.7489177489177489, Global best: 0.7489177489177489, Runtime: 1.73174 seconds
2023/09/10 10:43:13 PM, WARNING, mealpy.swarm_based.SHO.OriginalSHO: Stopping criterion wi

Solution: [90.46468276  0.        ], Fitness: 0.7489177489177489


# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import LabelEncoder

from mealpy.swarm_based.PSO import OriginalPSO
from mealpy.swarm_based.SHO import OriginalSHO

In [90]:
# Data we gonna use

tic_tac_toe_data = 'https://archive.ics.uci.edu/ml/machine-learning-databases/tic-tac-toe/tic-tac-toe.data' # 957 * 10
diabetes_data = 'diabetes.csv'  # (768, 9)
credit_card_data = 'creditcard.csv'  # 

In [ ]:
pd.read_csv(credit_card_data).shape

# 1 Parameter Tunning for KNN

# Our Approach with Tic_tac_toe_data

In [46]:
# Data preparation 
df = pd.read_csv(tic_tac_toe_data)

# Encode the categorical features
le = LabelEncoder()
for col in df.columns:
    df[col] = le.fit_transform(df[col])

# Split the dataset into training and testing sets
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
# Fiteness fuction for the optimization
def fitness_function(position):
    n_neighbors = int(position[0])
    weights = ['uniform', 'distance'][int(position[1])]
    clf = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    return score

# Define the problem as a single-objective maximization problem with two variables: N and w
problem_size = 2
lb = [1, 0]  # lower bounds for N and w
ub = [10, 1]  # upper bounds for N and w
problem = {
    "fit_func": fitness_function,
    "lb": [2, 0],
    "ub": [10, 1],
    "minmax": "max",
}

# Termination Function
terms = {"max_early_stop": 50}


In [48]:
# PSO
# Define the fitness function for the optimization problem

epoch = 100
pop_size = 30

model = OriginalPSO(epoch, pop_size)
best_position, best_fitness = model.solve(problem, termination=terms)
print(f"Solution: {best_position}, Fitness: {best_fitness}")

2023/09/10 10:48:31 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: Solving single objective optimization problem.
2023/09/10 10:48:32 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 1, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.27063 seconds
2023/09/10 10:48:32 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 2, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.27185 seconds
2023/09/10 10:48:32 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 3, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.24809 seconds
2023/09/10 10:48:33 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 4, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.27853 seconds
2023/09/10 10:48:33 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 5, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime

2023/09/10 10:48:45 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 46, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.27923 seconds
2023/09/10 10:48:45 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 47, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.28568 seconds
2023/09/10 10:48:45 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 48, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.30816 seconds
2023/09/10 10:48:46 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 49, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.27000 seconds
2023/09/10 10:48:46 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 50, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.25132 seconds
2023/09/10 10:48:46 PM, WARNING, mealpy.swarm_based.PSO.OriginalPSO: Stopping criterion wi

Solution: [6.36319906 0.42670771], Fitness: 0.8333333333333334


In [51]:
n_neighbors = int(best_position[0])
weights = ['uniform', 'distance'][int(best_position[1])]
clf = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)
clf.fit(X_train, y_train)
print("Score using PSO: ",clf.score(X_test, y_test))

Score using PSO:  0.8333333333333334


In [52]:
# Spotted Hyena Optimization

epoch = 100
pop_size = 50
h_factor = 5.0
N_tried = 10

model = OriginalSHO(epoch, pop_size, h_factor, N_tried)
best_position, best_fitness = model.solve(problem, termination=term_dict)
print(f"Solution: {best_position}, Fitness: {best_fitness}")

2023/09/10 10:51:46 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: Solving single objective optimization problem.
2023/09/10 10:51:48 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 1, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 1.23451 seconds
2023/09/10 10:51:49 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 2, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 1.05662 seconds
2023/09/10 10:51:50 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 3, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 1.18104 seconds
2023/09/10 10:51:51 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 4, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.95695 seconds
2023/09/10 10:51:52 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 5, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime

2023/09/10 10:52:38 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 46, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 1.01725 seconds
2023/09/10 10:52:39 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 47, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 1.07049 seconds
2023/09/10 10:52:41 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 48, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 1.95030 seconds
2023/09/10 10:52:43 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 49, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 1.69989 seconds
2023/09/10 10:52:45 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 50, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 1.65861 seconds
2023/09/10 10:52:45 PM, WARNING, mealpy.swarm_based.SHO.OriginalSHO: Stopping criterion wi

Solution: [8.36779407 0.88289705], Fitness: 0.8333333333333334


In [53]:
#SHO
n_neighbors = int(best_position[0])
weights = ['uniform', 'distance'][int(best_position[1])]
clf = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)
clf.fit(X_train, y_train)
print("Score using Sho: ",clf.score(X_test, y_test))

Score using Sho:  0.8333333333333334


### So there is no convergence when using PSO and SHO for Parameter tunning for KNN classification model. This is because the parameter we are tunning are only two and one of them is binary label and another is n_neighbours which will be low as property of KNN. Also the dataset size is very small, so low chances of optimization
### SO lets take other dataset and try tunning the parameters.

---
---

# Our Approach Data 2 : diabetes data

In [64]:
df = pd.read_csv(diabetes_data)
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [65]:
# Data preparation 
df = pd.read_csv(tic_tac_toe_data)

# Encode the categorical features
le = LabelEncoder()
for col in df.columns:
    df[col] = le.fit_transform(df[col])

# Split the dataset into training and testing sets
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [71]:
X_train

,x,x.1,x.2,x.3,o,o.1,x.4,o.2,o.3
302,1,2,2,1,2,0,2,1,0
467,1,0,1,1,0,2,2,2,2
294,1,2,2,2,2,1,2,1,1
548,0,1,2,1,0,1,2,2,2
465,1,0,1,2,0,1,2,2,2
...,...,...,...,...,...,...,...,...,...
106,2,2,0,1,2,1,0,2,1
270,2,0,1,0,2,1,0,0,2
860,1,0,2,1,1,2,2,2,1
435,1,0,2,1,2,0,2,2,1


In [66]:
# Fiteness fuction for the optimization
def fitness_function(position):
    n_neighbors = int(position[0])
    weights = ['uniform', 'distance'][int(position[1])]
    clf = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    return score

# Define the problem as a single-objective maximization problem with two variables: N and w
problem_size = 2
lb = [1, 0]  # lower bounds for N and w
ub = [10, 1]  # upper bounds for N and w
problem = {
    "fit_func": fitness_function,
    "lb": [2, 0],
    "ub": [20, 1],
    "minmax": "max",
}

# Termination Function
terms = {"max_early_stop": 50}


In [67]:
# PSO
# Define the fitness function for the optimization problem

epoch = 100
pop_size = 30

model = OriginalPSO(epoch, pop_size)
best_position, best_fitness = model.solve(problem, termination=terms)
print(f"Solution: {best_position}, Fitness: {best_fitness}")

2023/09/10 10:58:50 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: Solving single objective optimization problem.
2023/09/10 10:58:51 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 1, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.25522 seconds
2023/09/10 10:58:51 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 2, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.27424 seconds
2023/09/10 10:58:51 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 3, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.26928 seconds
2023/09/10 10:58:52 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 4, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.30572 seconds
2023/09/10 10:58:52 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 5, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime

2023/09/10 10:59:03 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 46, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.27451 seconds
2023/09/10 10:59:04 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 47, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.33139 seconds
2023/09/10 10:59:04 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 48, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.25924 seconds
2023/09/10 10:59:04 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 49, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.25778 seconds
2023/09/10 10:59:04 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 50, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 0.27079 seconds
2023/09/10 10:59:04 PM, WARNING, mealpy.swarm_based.PSO.OriginalPSO: Stopping criterion wi

Solution: [6.24045719 0.14838095], Fitness: 0.8333333333333334


In [68]:
n_neighbors = int(best_position[0])
weights = ['uniform', 'distance'][int(best_position[1])]
clf = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)
clf.fit(X_train, y_train)
print("Score using PSO: ",clf.score(X_test, y_test))

Score using PSO:  0.8333333333333334


In [69]:
# Spotted Hyena Optimization

epoch = 100
pop_size = 50
h_factor = 5.0
N_tried = 10

model = OriginalSHO(epoch, pop_size, h_factor, N_tried)
best_position, best_fitness = model.solve(problem, termination=term_dict)
print(f"Solution: {best_position}, Fitness: {best_fitness}")

2023/09/10 10:59:16 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: Solving single objective optimization problem.
2023/09/10 10:59:19 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 1, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 1.64647 seconds
2023/09/10 10:59:21 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 2, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 2.16945 seconds
2023/09/10 10:59:22 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 3, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 1.40487 seconds
2023/09/10 10:59:24 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 4, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 2.05289 seconds
2023/09/10 10:59:26 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 5, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime

2023/09/10 11:01:07 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 46, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 2.41192 seconds
2023/09/10 11:01:09 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 47, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 2.02493 seconds
2023/09/10 11:01:12 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 48, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 2.92713 seconds
2023/09/10 11:01:14 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 49, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 1.89558 seconds
2023/09/10 11:01:16 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 50, Current best: 0.8333333333333334, Global best: 0.8333333333333334, Runtime: 2.06264 seconds
2023/09/10 11:01:16 PM, WARNING, mealpy.swarm_based.SHO.OriginalSHO: Stopping criterion wi

Solution: [6.39680517 0.09025782], Fitness: 0.8333333333333334


In [70]:
#SHO
n_neighbors = int(best_position[0])
weights = ['uniform', 'distance'][int(best_position[1])]
clf = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)
clf.fit(X_train, y_train)
print("Score using Sho: ",clf.score(X_test, y_test))

Score using Sho:  0.8333333333333334


In [ ]:
-----------------------------------------------

# 1 Parameter Tunning for LGBMClassifier

## Diabetes data

In [77]:
df = pd.read_csv('diabetes.csv')

# Split the dataset into training and testing sets
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [78]:
from lightgbm import LGBMClassifier

# Define the fitness function for the optimization problem
def fitness_function(position):
    n_estimators = int(position[0])
    n_leaves = int(position[1])
    learning_rate = float(position[2])
    max_depth = int(position[3])
    clf = LGBMClassifier(n_estimators = n_estimators, n_leaves = n_leaves, learning_rate = learning_rate, max_depth = max_depth)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    return score

problem_size = 4
problem = {
    "fit_func": fitness_function,
    "lb": [10, 30, 0, -1],
    "ub": [500, 100, 1, 10],
    "minmax": "max",
}

# Termination Function
term_dict = {"max_early_stop": 50}

In [20]:
# PSO
epoch = 100
pop_size = 50

model = OriginalPSO(epoch, pop_size)
best_position, best_fitness = model.solve(problem, termination=term_dict)
print(f"Solution: {best_position}, Fitness: {best_fitness}")

2023/09/10 09:04:13 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: Solving single objective optimization problem.


[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning]

2023/09/10 09:04:17 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 1, Current best: 0.7662337662337663, Global best: 0.7662337662337663, Runtime: 1.49423 seconds


[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning]

2023/09/10 09:04:19 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 2, Current best: 0.7705627705627706, Global best: 0.7705627705627706, Runtime: 1.99856 seconds


...

[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning]

2023/09/10 09:09:43 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 75, Current best: 0.7792207792207793, Global best: 0.7792207792207793, Runtime: 3.86411 seconds


[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning]

2023/09/10 09:09:47 PM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 76, Current best: 0.7792207792207793, Global best: 0.7792207792207793, Runtime: 3.86908 seconds
2023/09/10 09:09:47 PM, WARNING, mealpy.swarm_based.PSO.OriginalPSO: Stopping criterion with early stopping (ES) (fitness-based) occurred. End program!


Solution: [2.62133926e+01 4.90711576e+01 2.95775194e-02 7.06413733e+00], Fitness: 0.7792207792207793


In [21]:
print(f"PSO Solution: {best_position}, Fitness: {best_fitness}")

PSO Solution: [2.62133926e+01 4.90711576e+01 2.95775194e-02 7.06413733e+00], Fitness: 0.7792207792207793


In [82]:
# SHO
from mealpy.swarm_based.SHO import OriginalSHO
problem = {
    "fit_func": fitness_function,
    "lb": [10, 30, 0.0001, -1],
    "ub": [500, 100, 1, 10],
    "minmax": "max",
}

# Spotted Hyena Optimization
epoch = 100
pop_size = 50
h_factor = 5.0
N_tried = 4.0
model = OriginalSHO(epoch, pop_size, h_factor, N_tried)
best_position, best_fitness = model.solve(problem, termination=term_dict)
print(f"Solution: {best_position}, Fitness: {best_fitness}")

2023/09/10 11:07:43 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: Solving single objective optimization problem.


[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning]

[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning]

2023/09/10 11:07:53 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 1, Current best: 0.7662337662337663, Global best: 0.7662337662337663, Runtime: 8.11036 seconds


[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning]

...

[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning]

2023/09/10 11:15:54 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 49, Current best: 0.7662337662337663, Global best: 0.7662337662337663, Runtime: 11.29020 seconds


[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning]

2023/09/10 11:16:03 PM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 50, Current best: 0.7662337662337663, Global best: 0.7662337662337663, Runtime: 9.18158 seconds
2023/09/10 11:16:03 PM, WARNING, mealpy.swarm_based.SHO.OriginalSHO: Stopping criterion with early stopping (ES) (fitness-based) occurred. End program!


Solution: [23.00493839 94.98597036  0.53078876  1.84267882], Fitness: 0.7662337662337663


In [84]:
print(f"Solution: {best_position}, Fitness: {best_fitness}")

Solution: [23.00493839 94.98597036  0.53078876  1.84267882], Fitness: 0.7662337662337663


In [ ]:
--------------------------------------------

In [86]:
# SSA
from mealpy.swarm_based.SSA import BaseSSA
term_dict = {"max_early_stop": 20}
model = BaseSSA(epoch, pop_size)
best_position, best_fitness = model.solve(problem, termination=term_dict)
print(f"Solution: {best_position}, Fitness: {best_fitness}")

2023/09/10 11:17:03 PM, INFO, mealpy.swarm_based.SSA.BaseSSA: Solving single objective optimization problem.


[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning]

2023/09/10 11:17:14 PM, INFO, mealpy.swarm_based.SSA.BaseSSA: >Problem: P, Epoch: 1, Current best: 0.7748917748917749, Global best: 0.7748917748917749, Runtime: 8.47916 seconds


[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning]

2023/09/10 11:17:26 PM, INFO, mealpy.swarm_based.SSA.BaseSSA: >Problem: P, Epoch: 2, Current best: 0.7748917748917749, Global best: 0.7748917748917749, Runtime: 12.34188 seconds


...

[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning]

2023/09/10 11:21:29 PM, INFO, mealpy.swarm_based.SSA.BaseSSA: >Problem: P, Epoch: 26, Current best: 0.7792207792207793, Global best: 0.7792207792207793, Runtime: 7.72450 seconds


[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning] Unknown parameter: n_leaves
[LightGBM] [Warning]

2023/09/10 11:21:37 PM, INFO, mealpy.swarm_based.SSA.BaseSSA: >Problem: P, Epoch: 27, Current best: 0.7792207792207793, Global best: 0.7792207792207793, Runtime: 8.22972 seconds
2023/09/10 11:21:37 PM, WARNING, mealpy.swarm_based.SSA.BaseSSA: Stopping criterion with early stopping (ES) (fitness-based) occurred. End program!


Solution: [8.09581149e+01 7.21715007e+01 3.27704235e-02 3.97493972e+00], Fitness: 0.7792207792207793


## Creditcard Data

In [143]:
df = pd.read_csv('machine_fail.csv')
df.drop('id',axis = 1,inplace=True)
df.columns = ['Air temperature','Process temperature','Rotational speed','Torque','Tool wear','Machine failure']
df

,Air temperature,Process temperature,Rotational speed,Torque,Tool wear,Machine failure
0,300.6,309.6,1596,36.1,140,0
1,302.6,312.1,1759,29.1,200,0
2,299.3,308.5,1805,26.5,25,0
3,301.0,310.9,1524,44.3,197,0
4,298.0,309.0,1641,35.4,34,0
...,...,...,...,...,...,...
136424,300.1,311.4,1530,37.5,210,0
136425,297.5,308.5,1447,49.1,2,0
136426,300.5,311.8,1524,38.5,214,0
136427,301.7,310.9,1447,46.3,42,0


In [144]:
clf = LGBMClassifier() #n_estimators = n_estimators, learning_rate = learning_rate, max_depth = max_depth) #n_leaves = n_leaves,
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
logloss = log_loss(y_test, y_pred)
logloss

In [145]:
score

0.9858069388299053

In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136429 entries, 0 to 136428
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Air temperature      136429 non-null  float64
 1   Process temperature  136429 non-null  float64
 2   Rotational speed     136429 non-null  int64  
 3   Torque               136429 non-null  float64
 4   Tool wear            136429 non-null  int64  
 5   Machine failure      136429 non-null  int64  
dtypes: float64(3), int64(3)
memory usage: 6.2 MB


In [128]:
# df = pd.read_csv(credit_card_data)
# df = df[:100]
# df

In [137]:
# Split the dataset into training and testing sets
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [139]:
# from lightgbm import LGBMClassifier
from sklearn.metrics import log_loss
# Define the fitness function for the optimization problem
def fitness_function(position):
    n_estimators = int(position[0])
    r_alpha = int(position[1])
    r_lambda = int(position[2])
    learning_rate = float(position[3])
    max_depth = int(position[4])
    feature_fraction = float(position[5])
    clf = LGBMClassifier(n_estimators = n_estimators, learning_rate = learning_rate, max_depth = max_depth) #n_leaves = n_leaves,
    clf.fit(X_train, y_train)
#     score = clf.score(X_test, y_test)
    y_pred = clf.predict(X_test)
    logloss = log_loss(y_test, y_pred)

    return logloss

problem_size = 4
problem = {
    "fit_func": fitness_function,
    "lb": [10, 0.0, 0.0, 0.001, -1, 0.5],
    "ub": [1000, 0.1, 0.1, 1, 10, 1],
    "minmax": "min",
}

# Termination Function
term_dict = {"max_early_stop": 10}

In [140]:
# PSO
epoch = 100
pop_size = 25

model = OriginalPSO(epoch, pop_size)
best_position, best_fitness = model.solve(problem, termination=term_dict)
print(f"Solution: {best_position}, Fitness: {best_fitness}")

2023/09/11 12:26:24 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: Solving single objective optimization problem.
2023/09/11 12:26:35 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 1, Current best: 0.9860956865532406, Global best: 0.9860956865532406, Runtime: 4.79880 seconds
2023/09/11 12:26:41 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 2, Current best: 0.9862289547332416, Global best: 0.9862289547332416, Runtime: 5.53587 seconds
2023/09/11 12:26:50 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 3, Current best: 0.9862289547332416, Global best: 0.9862289547332416, Runtime: 9.40936 seconds
2023/09/11 12:26:56 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 4, Current best: 0.9862289547332416, Global best: 0.9862289547332416, Runtime: 6.18591 seconds
2023/09/11 12:27:05 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 5, Current best: 0.986251166096575, Global best: 0.986251166096575, Runtime: 

Solution: [87.00469836  0.26014737  3.53265553], Fitness: 0.986251166096575


In [141]:
print(f"Solution with PSO: {best_position}, Fitness: {best_fitness}")

Solution with PSO: [87.00469836  0.26014737  3.53265553], Fitness: 0.986251166096575


In [142]:
# SHO

# Spotted Hyena Optimization
epoch = 100
pop_size = 50
h_factor = 5.0
N_tried = 4.0

model = OriginalSHO(epoch, pop_size)#, h_factor, N_tried)
best_position, best_fitness = model.solve(problem, termination=term_dict)
print(f"Solution: {best_position}, Fitness: {best_fitness}")

2023/09/11 12:28:27 AM, INFO, mealpy.swarm_based.SHO.OriginalSHO: Solving single objective optimization problem.
2023/09/11 12:28:51 AM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 1, Current best: 0.9860734751899072, Global best: 0.9860734751899072, Runtime: 21.54730 seconds
2023/09/11 12:29:14 AM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 2, Current best: 0.9860734751899072, Global best: 0.9860734751899072, Runtime: 22.48608 seconds
2023/09/11 12:29:36 AM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 3, Current best: 0.9860734751899072, Global best: 0.9860734751899072, Runtime: 21.64784 seconds
2023/09/11 12:30:07 AM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 4, Current best: 0.9860734751899072, Global best: 0.9860734751899072, Runtime: 31.06479 seconds
2023/09/11 12:30:35 AM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 5, Current best: 0.9860734751899072, Global best: 0.9860734751899072, Run

Solution: [19.29421523  0.1         4.28184693], Fitness: 0.9860956865532406


In [114]:
print(f"Solution with SHO : {best_position}, Fitness: {best_fitness}")

Solution with SHO : [51.88873578 38.21487521  0.18508     4.35193365], Fitness: 1.0


---

# Using log_loss 

In [162]:
df = pd.read_csv('pharyngitis.csv')
# df.drop('id',axis = 1,inplace=True)
# df.columns = ['Air temperature','Process temperature','Rotational speed','Torque','Tool wear','Machine failure']
df

,number,age_y,radt,pain,swollenadp,tender,tonsillarswelling,exudate,temperature,sudden,cough,rhinorrhea,conjunctivitis,headache,erythema,petechiae,abdopain,diarrhea,nauseavomit,scarlet
0,2000,7.0,0,1.0,1.0,0.0,0.0,0.0,37.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,2001,11.7,1,1.0,1.0,0.0,0.0,0.0,38.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2002,5.6,1,1.0,2.0,1.0,1.0,0.0,37.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,2003,6.0,1,1.0,2.0,1.0,1.0,0.0,40.8,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2004,5.2,0,1.0,2.0,0.0,1.0,0.0,40.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,3283,4.7,1,0.0,1.0,0.0,1.0,0.0,39.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
672,3284,3.3,0,1.0,0.0,0.0,1.0,1.0,38.5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
673,3285,4.6,1,1.0,1.0,0.0,1.0,0.0,39.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
674,3286,3.4,0,1.0,2.0,NaN,1.0,0.0,39.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,NaN


In [163]:
# Split the dataset into training and testing sets
y = df['radt']
X = df.drop('radt',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [166]:
from sklearn.metrics import log_loss
# Define the fitness function for the optimization problem
def fitness_function(position):
    n_estimators = int(position[0])
    r_alpha = int(position[1])
    r_lambda = int(position[2])
    learning_rate = float(position[3])
    max_depth = int(position[4])
    feature_fraction = float(position[5])
    clf = LGBMClassifier(n_estimators = n_estimators, learning_rate = learning_rate, max_depth = max_depth) #n_leaves = n_leaves,
    clf.fit(X_train, y_train)
#     score = clf.score(X_test, y_test)
    y_pred = clf.predict(X_test)
    logloss = log_loss(y_test, y_pred)

    return logloss

problem_size = 4
problem = {
    "fit_func": fitness_function,
    "lb": [10, 0.0, 0.0, 0.01, 0, 0.5],
    "ub": [100, 0.1, 0.1, 1, 10, 1],
    "minmax": "min",
}

# Termination Function
term_dict = {"max_early_stop": 50}

In [167]:
epoch = 100
pop_size = 30

model = OriginalPSO(epoch, pop_size)
best_position, best_fitness = model.solve(problem, termination=term_dict)
print(f"Solution: {best_position}, Fitness: {best_fitness}")

2023/09/11 03:17:46 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: Solving single objective optimization problem.
2023/09/11 03:17:47 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 1, Current best: 10.330883966048082, Global best: 10.330883966048082, Runtime: 0.40667 seconds
2023/09/11 03:17:47 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 2, Current best: 10.330883966048082, Global best: 10.330883966048082, Runtime: 0.43049 seconds
2023/09/11 03:17:48 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 3, Current best: 10.330876826785312, Global best: 10.330876826785312, Runtime: 0.46700 seconds
2023/09/11 03:17:48 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 4, Current best: 10.330876826785312, Global best: 10.330876826785312, Runtime: 0.47008 seconds
2023/09/11 03:17:49 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 5, Current best: 10.330876826785312, Global best: 10.330876826785312, Runtime

2023/09/11 03:18:39 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 46, Current best: 10.176678721473836, Global best: 10.176678721473836, Runtime: 2.15990 seconds
2023/09/11 03:18:41 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 47, Current best: 10.176678721473836, Global best: 10.176678721473836, Runtime: 1.38834 seconds
2023/09/11 03:18:42 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 48, Current best: 10.176678721473836, Global best: 10.176678721473836, Runtime: 1.17785 seconds
2023/09/11 03:18:43 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 49, Current best: 10.176678721473836, Global best: 10.176678721473836, Runtime: 1.51633 seconds
2023/09/11 03:18:45 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 50, Current best: 10.176678721473836, Global best: 10.176678721473836, Runtime: 1.31543 seconds
2023/09/11 03:18:46 AM, INFO, mealpy.swarm_based.PSO.OriginalPSO: >Problem: P, Epoch: 51, 

Solution: [17.95683148  0.04816125  0.04948243  0.52785894  1.99986997  0.62466406], Fitness: 10.176678721473836


In [168]:
print(f"Solution: {best_position}, Fitness: {best_fitness}")

Solution: [17.95683148  0.04816125  0.04948243  0.52785894  1.99986997  0.62466406], Fitness: 10.176678721473836


In [170]:
# SHO

# Spotted Hyena Optimization
# epoch = 100
pop_size = 50
h_factor = 5.0
N_tried = 4.0

model = OriginalSHO(epoch, pop_size, h_factor, N_tried)
best_position, best_fitness = model.solve(problem, termination=term_dict)
print(f"Solution: {best_position}, Fitness: {best_fitness}")

2023/09/11 03:20:23 AM, INFO, mealpy.swarm_based.SHO.OriginalSHO: Solving single objective optimization problem.
2023/09/11 03:20:26 AM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 1, Current best: 10.793439016037285, Global best: 10.793439016037285, Runtime: 2.32906 seconds
2023/09/11 03:20:33 AM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 2, Current best: 10.639240910725805, Global best: 10.639240910725805, Runtime: 6.89813 seconds
2023/09/11 03:20:40 AM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 3, Current best: 10.639240910725805, Global best: 10.639240910725805, Runtime: 6.62491 seconds
2023/09/11 03:20:45 AM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 4, Current best: 10.639240910725805, Global best: 10.639240910725805, Runtime: 5.27954 seconds
2023/09/11 03:20:51 AM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 5, Current best: 10.639240910725805, Global best: 10.639240910725805, Runtime

2023/09/11 03:25:08 AM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 46, Current best: 10.639240910725805, Global best: 10.639240910725805, Runtime: 7.09540 seconds
2023/09/11 03:25:15 AM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 47, Current best: 10.639240910725805, Global best: 10.639240910725805, Runtime: 6.96841 seconds
2023/09/11 03:25:21 AM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 48, Current best: 10.639240910725805, Global best: 10.639240910725805, Runtime: 5.55801 seconds
2023/09/11 03:25:26 AM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 49, Current best: 10.639240910725805, Global best: 10.639240910725805, Runtime: 4.93789 seconds
2023/09/11 03:25:34 AM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 50, Current best: 10.639240910725805, Global best: 10.639240910725805, Runtime: 8.45080 seconds
2023/09/11 03:25:41 AM, INFO, mealpy.swarm_based.SHO.OriginalSHO: >Problem: P, Epoch: 51, 

Solution: [20.37553062  0.          0.          0.06036754  3.47781214  0.51932357], Fitness: 10.639240910725805


In [161]:
print(f"Solution: {best_position}, Fitness: {best_fitness}")

Solution: [1.85798944e+02 1.37268852e-01 1.65394600e+00], Fitness: 0.6964285714285714


In [171]:
model = BaseSSA(epoch, pop_size)
best_position, best_fitness = model.solve(problem, termination=term_dict)
print(f"Solution: {best_position}, Fitness: {best_fitness}")

2023/09/11 03:26:46 AM, INFO, mealpy.swarm_based.SSA.BaseSSA: Solving single objective optimization problem.
2023/09/11 03:26:48 AM, INFO, mealpy.swarm_based.SSA.BaseSSA: >Problem: P, Epoch: 1, Current best: 10.176682291105221, Global best: 10.176682291105221, Runtime: 1.15138 seconds
2023/09/11 03:26:49 AM, INFO, mealpy.swarm_based.SSA.BaseSSA: >Problem: P, Epoch: 2, Current best: 10.176682291105221, Global best: 10.176682291105221, Runtime: 0.97168 seconds
2023/09/11 03:26:50 AM, INFO, mealpy.swarm_based.SSA.BaseSSA: >Problem: P, Epoch: 3, Current best: 10.176682291105221, Global best: 10.176682291105221, Runtime: 1.22497 seconds
2023/09/11 03:26:51 AM, INFO, mealpy.swarm_based.SSA.BaseSSA: >Problem: P, Epoch: 4, Current best: 9.86830392863919, Global best: 9.86830392863919, Runtime: 1.17016 seconds
2023/09/11 03:26:54 AM, INFO, mealpy.swarm_based.SSA.BaseSSA: >Problem: P, Epoch: 5, Current best: 9.86830392863919, Global best: 9.86830392863919, Runtime: 2.76679 seconds
2023/09/11 03:

2023/09/11 03:30:11 AM, INFO, mealpy.swarm_based.SSA.BaseSSA: >Problem: P, Epoch: 48, Current best: 9.868300359007804, Global best: 9.868300359007804, Runtime: 8.65326 seconds
2023/09/11 03:30:19 AM, INFO, mealpy.swarm_based.SSA.BaseSSA: >Problem: P, Epoch: 49, Current best: 9.868300359007804, Global best: 9.868300359007804, Runtime: 8.53787 seconds
2023/09/11 03:30:29 AM, INFO, mealpy.swarm_based.SSA.BaseSSA: >Problem: P, Epoch: 50, Current best: 9.868300359007804, Global best: 9.868300359007804, Runtime: 9.29346 seconds
2023/09/11 03:30:37 AM, INFO, mealpy.swarm_based.SSA.BaseSSA: >Problem: P, Epoch: 51, Current best: 9.868300359007804, Global best: 9.868300359007804, Runtime: 8.41749 seconds
2023/09/11 03:30:44 AM, INFO, mealpy.swarm_based.SSA.BaseSSA: >Problem: P, Epoch: 52, Current best: 9.868300359007804, Global best: 9.868300359007804, Runtime: 7.28962 seconds
2023/09/11 03:30:53 AM, INFO, mealpy.swarm_based.SSA.BaseSSA: >Problem: P, Epoch: 53, Current best: 9.868300359007804, G

Solution: [25.03563663  0.02633233  0.05330854  0.54113915  1.55697629  0.79775928], Fitness: 9.868300359007804
